In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [ ]:
cwd = os.getcwd()
print(cwd)
# cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
cwd_parent = os.path.abspath(os.path.join(cwd, '../../'))
print(cwd_parent)

sys.path.append(cwd_parent)

In [ ]:
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.chemfeatures import *
from deepadr.train_functions_gnngat import *
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [ ]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [ ]:
report_available_cuda_devices()

In [ ]:
n_gpu = torch.cuda.device_count()
n_gpu

In [ ]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

In [ ]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

### Preparing dataset 

In [ ]:
# options: 
# 'total_thresh' + 4,3,2
# 'loewe_thresh', 'hsa_thresh', 'bliss_thresh', 'zip_thresh' + 1

score = 'hsa_thresh'
score_val = 1

In [ ]:
DSdataset_name = f'DrugComb_{score}_{score_val}'

data_fname = 'data_v1' # v2 for baseline models

In [ ]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

In [ ]:
%%time

# Make sure to first run the "DDoS_Dataset_Generation" notebook first

dataset = MoleculeDataset(root=targetdata_dir)

In [ ]:
dataset.data.y

In [ ]:
fold_partitions = get_stratified_partitions(dataset.data.y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

In [ ]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))

In [ ]:
# print(dataset.num_classes)

In [ ]:
# training parameters

tp = {
    "batch_size" : 300,
    "num_epochs" : 100,
    
    "emb_dim" : 100,
    "gnn_type" : "gat",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-4, #3e-4
    "max_lr_mul": 10,
    "l2_reg" : 1e-7,
    "loss_w" : 1.,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 1024,
    "exp_H2" : 512
}

In [ ]:
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(queue, used_dataset, gpu_num, tphp, exp_dir, partition):
    return mp.Process(target=deepadr.train_functions_gnngat.run_exp_vanilla, args=(queue, used_dataset, gpu_num, tphp, exp_dir, partition))

In [ ]:
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

queue = mp.Queue()
q_processes = []

time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

print("Start: " + time_stamp)

for q_i in range(min(n_gpu, len(fold_partitions))):
    partition = fold_partitions[q_i]
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))
    
    q_process = create_q_process(queue, dataset, q_i, tp, exp_dir, partition)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
for q_i in range(min(n_gpu, len(fold_partitions))):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)

In [ ]:
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
